In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

SE

In [4]:
SE_students_data = pd.read_excel("data_v1.xlsx")
SE_students_data.head()

,STUDENT_NUMBER,COURSE_DESCRIPTION,INTAKE_CODE,MODULE_CODE,MODULE_NAME,GRADE,STUDY_MODE
0,ABC0001,MSc in Data Science and Business Analytics,UCMP1709DSBA,AQ049-3-M-MMDA,Multivariate Methods for Data Analysis,B+,Part time
1,ABC0001,MSc in Data Science and Business Analytics,UCMP1709DSBA,AQ050-3-M-TSAF,Time Series Analysis and Forecasting,C-,Part time
2,ABC0001,MSc in Data Science and Business Analytics,UCMP1709DSBA,AQ051-3-M-MDA,Multilevel Data Analysis,A,Part time
3,ABC0001,MSc in Data Science and Business Analytics,UCMP1709DSBA,AQ052-3-M-ORO,Operational Research and Optimisation,NaN,Part time
4,ABC0001,MSc in Data Science and Business Analytics,UCMP1709DSBA,BM026-3-M-SEM,Strategies in Emerging Markets,NaN,Part time


In [5]:
SE_students_data = SE_students_data[SE_students_data['COURSE_DESCRIPTION'] != 'MSc in Data Science and Business Analytics']
SE_students_data = SE_students_data.drop(["INTAKE_CODE", "MODULE_CODE", "STUDY_MODE"], axis=1)

module_name_changes = {
    'Managing Software Development Projects': 'MSDP',
    'Reliability Management': 'RM',
    'Object-Oriented Software Systems Engineering': 'OOSSE',
    'Object Oriented Software Systems Engineering': 'OOSSE',
    'Software Engineering Support Environments': 'SESE',
    'Software Quality Engineering': 'SQE',
    'Security Technologies': 'ST',
    'Research Methodology in Computing and Engineering': 'RMCE | RMCP',
    'Big Data Analytics and Technologies': 'BDAT',
    'Data Management': 'DM',
    'Internet Applications': 'IA',
    'Natural Language Processing': 'MDA | NLP',
    'Network Design and Performance': 'NDP'
    # 'Final Dissertation': 'Project',
    # 'Project Paper': 'Project',
    # 'Dissertation': 'Project',
    # 'MSc Dissertation': 'Project',
    # 'Research Paper': 'Project'
}

SE_students_data['MODULE_NAME'] = SE_students_data['MODULE_NAME'].replace(module_name_changes)

valid_modules = ['MSDP', 'RM', 'OOSSE', 'SESE', 'SQE', 'ST', 'RMCE | RMCP', 'BDAT', 'DM', 'IA', 'MDA | NLP', 'NDP']
SE_students_data = SE_students_data[SE_students_data['MODULE_NAME'].isin(valid_modules)]

In [6]:
duplicated_modules = SE_students_data[SE_students_data.duplicated(['STUDENT_NUMBER', 'MODULE_NAME'], keep=False)]
duplicated_modules

,STUDENT_NUMBER,COURSE_DESCRIPTION,MODULE_NAME,GRADE
413,ABC0106,MSc in Software Engineering,SESE,M
423,ABC0106,MSc in Software Engineering,SESE,NaN


In [7]:
duplicated_modules = SE_students_data[SE_students_data.duplicated(['STUDENT_NUMBER', 'MODULE_NAME'], keep=False)]

# Function to decide which row to keep based on 'GRADE'
def filter_grades(group):
    if group['GRADE'].notna().sum() > 0:
        return group[group['GRADE'].notna()].drop_duplicates(subset=['STUDENT_NUMBER', 'MODULE_NAME'], keep='first')
    else:
        return group.drop_duplicates(subset=['STUDENT_NUMBER', 'MODULE_NAME'], keep='first')

# Apply the function to each group
filtered_students = duplicated_modules.groupby(['STUDENT_NUMBER', 'MODULE_NAME']).apply(filter_grades).reset_index(drop=True)

# Combine with original DataFrame to get the final cleaned DataFrame
SE_students_data = pd.concat([SE_students_data.drop(index=duplicated_modules.index), filtered_students])

In [8]:
SE_students_data.reset_index(drop=True, inplace=True)

In [9]:
SE_students_data.head()

,STUDENT_NUMBER,COURSE_DESCRIPTION,MODULE_NAME,GRADE
0,ABC0013,MSc in Software Engineering,BDAT,NaN
1,ABC0013,MSc in Software Engineering,DM,B+
2,ABC0013,MSc in Software Engineering,RMCE | RMCP,C
3,ABC0013,MSc in Software Engineering,MSDP,A
4,ABC0013,MSc in Software Engineering,IA,B+


In [10]:
duplicated_modules = SE_students_data[SE_students_data.duplicated(['STUDENT_NUMBER', 'MODULE_NAME'], keep=False)]
print(duplicated_modules)

Empty DataFrame
Columns: [STUDENT_NUMBER, COURSE_DESCRIPTION, MODULE_NAME, GRADE]
Index: []


In [11]:
SE_each_student_info = SE_students_data.groupby('STUDENT_NUMBER').apply(
    lambda x:{'Modules': x['MODULE_NAME'].tolist(), 'Grades': x['GRADE'].tolist()}
).to_dict()

print(SE_each_student_info)

{'ABC0013': {'Modules': ['BDAT', 'DM', 'RMCE | RMCP', 'MSDP', 'IA', 'NDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE'], 'Grades': [nan, 'B+', 'C', 'A', 'B+', 'C-', 'B+', 'B', 'B+', 'C', 'C']}, 'ABC0074': {'Modules': ['MSDP', 'NDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE', 'RMCE | RMCP'], 'Grades': [nan, nan, nan, nan, nan, 'U', 'A', nan]}, 'ABC0099': {'Modules': ['BDAT', 'MDA | NLP', 'MSDP', 'IA', 'NDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE', 'RMCE | RMCP'], 'Grades': ['C+', 'A', 'B+', 'A', nan, 'B+', 'B+', 'B+', 'A', 'A+', 'A']}, 'ABC0106': {'Modules': ['OOSSE', 'SQE', 'RM', 'SESE'], 'Grades': ['D', 'M', 'D', 'M']}, 'ABC0109': {'Modules': ['BDAT', 'DM', 'MDA | NLP', 'MSDP', 'IA', 'NDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE', 'RMCE | RMCP'], 'Grades': ['A', 'B+', nan, 'B+', nan, nan, nan, nan, 'A', 'A', 'B', nan]}, 'ABC0110': {'Modules': ['BDAT', 'DM', 'MSDP', 'IA', 'NDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE', 'RMCE | RMCP'], 'Grades': ['A', nan, 'A+', 'A+', nan, 'A', nan, 'A', 'A+', 'A+', 'A']}, 'ABC0112'

In [12]:
incomplete_modules = {}
completed_modules = {}

for student_id, modules_info in SE_each_student_info.items():
    modules  = modules_info['Modules']
    grades = modules_info['Grades']

    incomplete_modules[student_id] = []
    completed_modules[student_id] = []
    
    for module, grade in zip(modules, grades):
        if pd.isna(grade):
            incomplete_modules[student_id].append(module)
        else:
            completed_modules[student_id].append(module)
            
print("Incomplete Modules:", incomplete_modules)
print("Completed Modules:", completed_modules)

Incomplete Modules: {'ABC0013': ['BDAT'], 'ABC0074': ['MSDP', 'NDP', 'OOSSE', 'RM', 'ST', 'RMCE | RMCP'], 'ABC0099': ['NDP'], 'ABC0106': [], 'ABC0109': ['MDA | NLP', 'IA', 'NDP', 'OOSSE', 'RM', 'RMCE | RMCP'], 'ABC0110': ['DM', 'NDP', 'RM'], 'ABC0112': ['DM', 'MDA | NLP', 'IA', 'NDP'], 'ABC0122': ['BDAT', 'MDA | NLP', 'MSDP', 'IA', 'NDP', 'OOSSE', 'ST', 'SESE', 'SQE', 'RMCE | RMCP'], 'ABC0145': ['DM', 'IA'], 'ABC0184': ['MSDP', 'IA', 'NDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE', 'RMCE | RMCP'], 'ABC0200': [], 'ABC0201': ['NDP', 'OOSSE'], 'ABC0202': ['MSDP', 'NDP'], 'ABC0210': ['MDA | NLP', 'IA'], 'ABC0219': [], 'ABC0225': ['BDAT'], 'ABC0266': [], 'ABC0274': ['IA'], 'ABC0277': ['IA'], 'ABC0291': ['NDP'], 'ABC0501': ['BDAT', 'NDP'], 'ABC0503': ['MDA | NLP', 'MSDP', 'IA', 'NDP', 'OOSSE', 'ST', 'SESE', 'RMCE | RMCP'], 'ABC0555': ['MDA | NLP', 'NDP']}
Completed Modules: {'ABC0013': ['DM', 'RMCE | RMCP', 'MSDP', 'IA', 'NDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE'], 'ABC0074': ['SESE', 'SQE'], 'ABC0

In [13]:
for student_id, module in incomplete_modules.items():
    print(student_id, module)

ABC0013 ['BDAT']
ABC0074 ['MSDP', 'NDP', 'OOSSE', 'RM', 'ST', 'RMCE | RMCP']
ABC0099 ['NDP']
ABC0106 []
ABC0109 ['MDA | NLP', 'IA', 'NDP', 'OOSSE', 'RM', 'RMCE | RMCP']
ABC0110 ['DM', 'NDP', 'RM']
ABC0112 ['DM', 'MDA | NLP', 'IA', 'NDP']
ABC0122 ['BDAT', 'MDA | NLP', 'MSDP', 'IA', 'NDP', 'OOSSE', 'ST', 'SESE', 'SQE', 'RMCE | RMCP']
ABC0145 ['DM', 'IA']
ABC0184 ['MSDP', 'IA', 'NDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE', 'RMCE | RMCP']
ABC0200 []
ABC0201 ['NDP', 'OOSSE']
ABC0202 ['MSDP', 'NDP']
ABC0210 ['MDA | NLP', 'IA']
ABC0219 []
ABC0225 ['BDAT']
ABC0266 []
ABC0274 ['IA']
ABC0277 ['IA']
ABC0291 ['NDP']
ABC0501 ['BDAT', 'NDP']
ABC0503 ['MDA | NLP', 'MSDP', 'IA', 'NDP', 'OOSSE', 'ST', 'SESE', 'RMCE | RMCP']
ABC0555 ['MDA | NLP', 'NDP']


In [14]:
elective = ["BDAT", "DM", "IA", "MDA | NLP", "NDP"]
for student_id, completed_modules_info in completed_modules.items():
    completed_elective = set(completed_modules_info) & set(elective)
    
    if len(completed_elective) == 3:
        for student_id, incomplete_modules_info in incomplete_modules.items():
            extra_elective = set(incomplete_modules_info) & set(elective)
            
            for module in extra_elective:
                completed_modules[student_id].append(module)
                incomplete_modules_info.remove(module)
                
print("Incomplete Modules:", incomplete_modules)
print("Completed Modules:", completed_modules)

Incomplete Modules: {'ABC0013': [], 'ABC0074': ['MSDP', 'OOSSE', 'RM', 'ST', 'RMCE | RMCP'], 'ABC0099': [], 'ABC0106': [], 'ABC0109': ['OOSSE', 'RM', 'RMCE | RMCP'], 'ABC0110': ['RM'], 'ABC0112': [], 'ABC0122': ['MSDP', 'OOSSE', 'ST', 'SESE', 'SQE', 'RMCE | RMCP'], 'ABC0145': [], 'ABC0184': ['MSDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE', 'RMCE | RMCP'], 'ABC0200': [], 'ABC0201': ['OOSSE'], 'ABC0202': ['MSDP'], 'ABC0210': [], 'ABC0219': [], 'ABC0225': [], 'ABC0266': [], 'ABC0274': [], 'ABC0277': [], 'ABC0291': [], 'ABC0501': [], 'ABC0503': ['MSDP', 'OOSSE', 'ST', 'SESE', 'RMCE | RMCP'], 'ABC0555': []}
Completed Modules: {'ABC0013': ['DM', 'RMCE | RMCP', 'MSDP', 'IA', 'NDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE', 'BDAT'], 'ABC0074': ['SESE', 'SQE', 'NDP'], 'ABC0099': ['BDAT', 'MDA | NLP', 'MSDP', 'IA', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE', 'RMCE | RMCP', 'NDP'], 'ABC0106': ['OOSSE', 'SQE', 'RM', 'SESE'], 'ABC0109': ['BDAT', 'DM', 'MSDP', 'ST', 'SESE', 'SQE', 'NDP', 'MDA | NLP', 'IA'], 'ABC0110':

In [15]:
for student_id, module in incomplete_modules.items():
    print(student_id, module)

ABC0013 []
ABC0074 ['MSDP', 'OOSSE', 'RM', 'ST', 'RMCE | RMCP']
ABC0099 []
ABC0106 []
ABC0109 ['OOSSE', 'RM', 'RMCE | RMCP']
ABC0110 ['RM']
ABC0112 []
ABC0122 ['MSDP', 'OOSSE', 'ST', 'SESE', 'SQE', 'RMCE | RMCP']
ABC0145 []
ABC0184 ['MSDP', 'OOSSE', 'RM', 'ST', 'SESE', 'SQE', 'RMCE | RMCP']
ABC0200 []
ABC0201 ['OOSSE']
ABC0202 ['MSDP']
ABC0210 []
ABC0219 []
ABC0225 []
ABC0266 []
ABC0274 []
ABC0277 []
ABC0291 []
ABC0501 []
ABC0503 ['MSDP', 'OOSSE', 'ST', 'SESE', 'RMCE | RMCP']
ABC0555 []
